In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# libraries that are generally imported for any deep learning model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from sklearn.metrics import confusion_matrix

# libraries need to prepare the data
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array

# libraries required to build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model

# libraries for activation functions required
from tensorflow.keras.activations import relu, softmax

# weight initializer libraries
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.initializers import HeUniform

# optimizer library
from tensorflow.keras.optimizers import Adam

# callback library
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# setting the random seed to generalize the output
# the seed value is used to generate the random number generator. 
# And, every time you use the same seed value, you will get the same random values.

# setting random seed in numpy
from numpy.random import seed
seed(1)

# setting random seed in tensorflow
tf.random.set_seed(2) 

In [3]:
train_data_gen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    rotation_range=2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    vertical_flip=True)

training_set = train_data_gen.flow_from_directory(directory="../../Datasets/BrainTumorSplitData_images/Training",
                                                  target_size=(224,224),
                                                  class_mode='categorical',
                                                  batch_size=32)

Found 8205 images belonging to 4 classes.


In [4]:
validation_data_gen = ImageDataGenerator(rescale=1./255)

validation_set = validation_data_gen.flow_from_directory(directory="../../Datasets/BrainTumorSplitData_images/Testing",
                                                        target_size=(224,224),
                                                        class_mode='categorical',
                                                        batch_size=32)

Found 1705 images belonging to 4 classes.


In [9]:
loaded = load_model('../TumorSense_app/fine_tuned_model.h5')


In [10]:
index = ['glioma','meningioma','notumor','pituitary']

def test_model(model):
    test_dataset_dir = '../../Datasets/BrainTumorSplitData_images/Testing'

    test_images = []
    test_labels = []
    
    for class_name in index:
        class_dir = os.path.join(test_dataset_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            test_image = load_img(image_path, target_size=(224, 224))
            test_image = img_to_array(test_image) / 255.0
            test_images.append(test_image)
            test_labels.append(class_name)
    
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    
    predictions = model.predict(test_images)
    
    predicted_labels = [index[np.argmax(prediction)] for prediction in predictions]
    
    accuracy = np.mean(predicted_labels == test_labels)
    print(f"Accuracy: {accuracy*100}%")


In [11]:
test_model(loaded)

54/54 [==============================] - 3s 45ms/step
Accuracy: 88.97360703812318%


In [14]:
loaded1 = load_model('../TumorSense_app/models/fine_tuned_model.h5')
loaded1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
index = ['glioma','meningioma','normal','adenoma']

test_image1 = load_img('../testing_dataset_limited/adenoma/Te-pi_0010.jpg',target_size = (224,224))

test_image1 = img_to_array(test_image1)
test_image1 = np.expand_dims(test_image1,axis=0)
result1 = np.argmax(loaded1.predict(test_image1/255.0),axis=1)
print(index[result1[0]])

1/1 [==============================] - 0s 21ms/step
adenoma
